## 1.集合標量操作

### first / reduce / count / collect

In [1]:
var rdd1 = sc.makeRDD(Array(("A","1"),("B","2"),("C","3")),2)

In [2]:
//first返回RDD中的第一個元素，不排序
rdd1.first

(A,1)

In [3]:
var rdd1 = sc.makeRDD(1 to 10, 2)

In [4]:
//reduce根據映射函數f，對RDD中的元素進行二元計算，返回計算結果。
rdd1.reduce(_+_)

55

In [5]:
var rdd1 = sc.makeRDD(Array(("A",0),("A",2),("B",1),("B",2),("C",1)),2)

In [6]:
rdd1.reduce((x,y) => {
        (x._1 + y._1,x._2 + y._2)
    }
)

(AABBC,6)

In [7]:
//count返回RDD中的元素數量。
rdd1.count

5

In [8]:
//collect用於將一個RDD轉換成陣列
rdd1.collect

Array((A,0), (A,2), (B,1), (B,2), (C,1))

### take / top / takeOrdered

In [9]:
var rdd1 = sc.makeRDD(Seq(10, 4, 2, 12, 3))

In [10]:
//take用於獲取RDD中從0到num-1下標的元素，不排序
rdd1.take(2)

Array(10, 4)

In [11]:
//top函數用於從RDD中，按照默認（降冪）或者指定的排序規則，返回前num個元素。
rdd1.top(2)

Array(12, 10)

In [12]:
//takeOrdered和top類似，只不過以和top相反的順序返回元素。
rdd1.takeOrdered(2)

Array(2, 3)

### aggregate / fold / lookup

In [13]:
var rdd1 = sc.makeRDD(1 to 10,2)

In [14]:
rdd1.mapPartitionsWithIndex{
        (partIdx,iter) => {
          var part_map = scala.collection.mutable.Map[String,List[Int]]()
            while(iter.hasNext){
              var part_name = "part_" + partIdx;
              var elem = iter.next()
              if(part_map.contains(part_name)) {
                var elems = part_map(part_name)
                elems ::= elem
                part_map(part_name) = elems
              } else {
                part_map(part_name) = List[Int]{elem}
              }
            }
            part_map.iterator
           
        }
      }.collect

Array((part_0,List(5, 4, 3, 2, 1)), (part_1,List(10, 9, 8, 7, 6)))

In [15]:
//aggregate使用者聚合RDD中的元素，先使用seqOp將RDD中每個分區中的T類型元素聚合成U類型，
//再使用combOp將之前每個分區聚合後的U類型聚合成U類型，特別注意seqOp和combOp都會使用zeroValue的值，
//zeroValue的類型為U。

rdd1.aggregate(1)(
    //seqOp
    {(x : Int,y : Int) => x + y}, 
    //combOp
    {(a : Int,b : Int) => a + b}
)

58

In [16]:
//結果是58，看下面的計算過程：
//先在每個分區中反覆運算執行 (x : Int,y : Int) => x + y 並且使用zeroValue的值1
//即：part_0中 zeroValue+5+4+3+2+1 = 1+5+4+3+2+1 = 16
//part_1中 zeroValue+10+9+8+7+6 = 1+10+9+8+7+6 = 41
//再將兩個分區的結果合併(a : Int,b : Int) => a + b ，並且使用zeroValue的值1
//即：zeroValue+part_0+part_1 = 1 + 16 + 41 = 58

Name: Syntax Error.
Message: 
StackTrace: 

In [17]:
rdd1.aggregate(2)(
    {(x : Int,y : Int) => x + y}, 
    {(a : Int,b : Int) => a * b}
)

1428

In [18]:
//這次zeroValue=2
//part_0中 zeroValue+5+4+3+2+1 = 2+5+4+3+2+1 = 17
//part_1中 zeroValue+10+9+8+7+6 = 2+10+9+8+7+6 = 42
//最後：zeroValue*part_0*part_1 = 2 * 17 * 42 = 1428

Name: Syntax Error.
Message: 
StackTrace: 

In [19]:
//fold是aggregate的簡化，將aggregate中的seqOp和combOp使用同一個函數op。
//結果同上面使用aggregate的第一個例子一樣
rdd1.fold(1)(
    (x,y) => x + y  
)

58

In [20]:
var rdd1 = sc.makeRDD(Array(("A",0),("A",2),("B",1),("B",2),("C",1)))

In [21]:
//lookup用于(K,V)类型的RDD,指定K值，返回RDD中该K对应的所有V值。
rdd1.lookup("A")

WrappedArray(0, 2)

In [22]:
rdd1.lookup("B")

WrappedArray(1, 2)

In [23]:
var rdd1 = sc.makeRDD(Array(("A",0),("A",2),("B",1),("B",2),("B",3)))

In [24]:
//countByKey用於統計RDD[K,V]中每個K的數量
rdd1.countByKey

Map(B -> 3, A -> 2)

In [25]:
var cnt = sc.accumulator(0)

In [26]:
var rdd1 = sc.makeRDD(1 to 10,2)

In [27]:
//foreach用於遍歷RDD,將函數f應用於每一個元素。
//但要注意，如果對RDD執行foreach，只會在Executor端有效，而並不是Driver端。
//比如：rdd.foreach(println)，只會在Executor的stdout中列印出來，Driver端是看不到的。
rdd1.foreach( x => cnt += x)

In [28]:
cnt.value

55

In [29]:
var rdd1 = sc.makeRDD(1 to 10,2)

In [30]:
var allsize = sc.accumulator(0)

In [31]:
//foreachPartition和foreach類似，只不過是對每一個分區使用f。
rdd1.foreachPartition { x => {
    allsize += x.size
}}

In [32]:
allsize.value

10

In [33]:
var rdd1 = sc.makeRDD(Seq(3,6,7,1,2,0),2)

In [34]:
//sortBy根據給定的排序k函數將RDD中的元素進行排序 default asc
rdd1.sortBy(x => x).collect

Array(0, 1, 2, 3, 6, 7)

In [35]:
//false desc
rdd1.sortBy(x => x, false).collect

Array(7, 6, 3, 2, 1, 0)

In [36]:
//RDD[K,V]類型
var rdd1 = sc.makeRDD(Array(("A",2),("A",1),("B",6),("B",3),("B",7)))

In [37]:
rdd1.sortBy(x=>x).collect

Array((A,1), (A,2), (B,3), (B,6), (B,7))

In [38]:
//按照V進行降冪排序
rdd1.sortBy(x=>x._2,false).collect

Array((B,7), (B,6), (B,3), (A,2), (A,1))